- #### Transformaciones
    - ##### Windows

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .appName("sesion_1") \
        .master("local[*]") \
        .getOrCreate()

In [ ]:
def read_csv(path):
    return spark.read\
        .option("header","true")\
        .option("delimiter",",")\
        .option("inferSchema","false")\
        .csv(path)

base_path = "../../resources/data/csv/"
contracts_df = read_csv(base_path + "contracts.csv")
contracts_df.show()

In [ ]:
# Windows
    # Agregacion -> max, min, count, avg, mean, sum, ...
    # Ranking -> row_number, rank, dense_rank, ...
    # lag lead

from pyspark.sql import Window
import pyspark.sql.functions as f

contracts_df.orderBy(f.col("cod_iuc")).show()

window_1 = Window.partitionBy(f.col("cod_iuc"))
window_2 = Window.partitionBy(f.col("cod_iuc")).orderBy(f.col("fec_alta").asc())


contracts_df.select(
    *contracts_df.columns,
    f.count("*").over(window_1).alias("count_w"),
    f.max(f.col("fec_alta")).over(window_2).alias("max_fec_alta_w"),
    f.sum(f.col("cod_producto")).over(window_2).alias("sum_cod_producto_w")
).show()

In [ ]:
contracts_df.orderBy(f.col("cod_iuc")).show()

window_3 = Window.partitionBy(f.col("cod_iuc")).orderBy(f.col("cod_titular").asc())


contracts_df.select(
    *contracts_df.columns,
    f.row_number().over(window_3).alias("row_number"),
    f.rank().over(window_3).alias("rank"),
    f.dense_rank().over(window_3).alias("dense_rank")
).show(100)

In [ ]:
contracts_df.orderBy(f.col("cod_iuc")).show()

window_4 = Window.partitionBy(f.col("cod_iuc")).orderBy(f.col("fec_alta").desc())


contracts_df.select(
    *contracts_df.columns,
    f.lag(f.col("fec_alta"), 1, "1970-01-01").over(window_4).alias("lag"),
    f.lead(f.col("fec_alta"), 1, "1970-01-01").over(window_4).alias("lead")
).show(100)